In [9]:
import pandas as pd
import requests
from pprint import pprint as pp

In [32]:
url_list = []
for year in range(2019,2020):
    url = "http://www.nfl.com/feeds-rs/schedules/{}.json".format(year)
    game_id_url = "http://www.nfl.com/feeds-rs/scores/byGame/"
    response = requests.get(url)
    results = response.json()
    games = results['gameSchedules']
    game_ids = []
    game_ids = ["{}{}.json".format(game_id_url, item[-1]) for game in games for item in game.items() if item[0] == "gameId"]
    url_list.extend(game_ids)

In [34]:
def parse_data(url):
    row = []
    g_response = requests.get(url)
    g_results = g_response.json()
    for sect in g_results.items():
        if sect[0] == 'gameSchedule':
            for item in sect[-1].items():
                if (item[0] == "visitorTeam"):
                    continue
                elif (item[0] == "homeTeam"):
                    for it in item[-1].items():
                        if it[0] == "teamId":
                            team_id = it[-1]
                elif (item[0] == "gameId"):
                    game_id = item[-1]
                elif (item[0] == "site"):
                    for it in item[-1].items():
                        row.append(it[-1])
                else:
                    row.append(item[-1])
        else:
            for item in sect[-1].items():
                if (item[0] == 'homeTeamScore') | (item[0] == 'visitorTeamScore'):
                    tot = item[-1].get('pointTotal')
                    row.append(tot)
        stat_url = "http://www.nfl.com/feeds-rs/teamGameStats/{}/{}.json".format(team_id, game_id)
    s_response = requests.get(stat_url)
    s_results = s_response.json()
    home_team_stat_line = s_results['teamStatDetail']
    visitor_team_stat_line = s_results['opponentStatDetail']
    teams_stats = [home_team_stat_line, visitor_team_stat_line]
    for team in teams_stats:
        for item in team.items():
            row.append(item[-1])
    return row

In [35]:
len(url_list)

321

In [37]:
games_list = []
for year in range(2019,2020):
    url = "http://www.nfl.com/feeds-rs/schedules/{}.json".format(year)
    game_id_url = "http://www.nfl.com/feeds-rs/scores/byGame/"
    response = requests.get(url)
    results = response.json()
    games = results['gameSchedules']
    game_ids = []
    game_ids = ["{}{}.json".format(game_id_url, item[-1]) for game in games for item in game.items() if item[0] == "gameId"]
    games_list.extend(game_ids)

In [38]:
len(games_list)

321

In [64]:
data = []
counter = 0
for game in games_list:
    row = []
    g_response = requests.get(game)
    g_results = g_response.json()
    g_schedule = g_results['gameSchedule']
    for item in g_schedule.items():
        if (item[0] == 'visitorTeam') | (item[0] == 'homeTeam') | (item[0] == 'site'):
            for it in item[-1].items():
                point = it[-1]
                row.append(point)
        else:
            row.append(item[-1])
    counter += 1        
    print("{} out of {}".format(counter, len(games_list)))
    data.append(row)

1 out of 321
2 out of 321
3 out of 321
4 out of 321
5 out of 321
6 out of 321
7 out of 321
8 out of 321
9 out of 321
10 out of 321
11 out of 321
12 out of 321
13 out of 321
14 out of 321
15 out of 321
16 out of 321
17 out of 321
18 out of 321
19 out of 321
20 out of 321
21 out of 321
22 out of 321
23 out of 321
24 out of 321
25 out of 321
26 out of 321
27 out of 321
28 out of 321
29 out of 321
30 out of 321
31 out of 321
32 out of 321
33 out of 321
34 out of 321
35 out of 321
36 out of 321
37 out of 321
38 out of 321
39 out of 321
40 out of 321
41 out of 321
42 out of 321
43 out of 321
44 out of 321
45 out of 321
46 out of 321
47 out of 321
48 out of 321
49 out of 321
50 out of 321
51 out of 321
52 out of 321
53 out of 321
54 out of 321
55 out of 321
56 out of 321
57 out of 321
58 out of 321
59 out of 321
60 out of 321
61 out of 321
62 out of 321
63 out of 321
64 out of 321
65 out of 321
66 out of 321
67 out of 321
68 out of 321
69 out of 321
70 out of 321
71 out of 321
72 out of 321
7

In [65]:
columns = []
g_response = requests.get(game)
g_results = g_response.json()
g_schedule = g_results['gameSchedule']
for item in g_schedule.items():
    if (item[0] == 'visitorTeam') | (item[0] == 'homeTeam') | (item[0] == 'site'):
        for it in item[-1].items():
            point = it[0]
            columns.append(point)
    else:
        columns.append(item[0])

In [69]:
df = pd.DataFrame(data)
df.columns = columns
# df.to_csv("../data/interim/game_data.csv", index=False)

In [70]:
df.head()

,season,seasonType,week,gameId,gameKey,gameDate,gameTimeEastern,gameTimeLocal,isoTime,homeTeamId,...,fullName,nick,teamType,conferenceAbbr,divisionAbbr,siteId,siteCity,siteFullname,siteState,roofType
0,2019,PRE,0,2019080151,57835,08/01/2019,20:00:00,20:00:00,1564704000000,0200,...,Atlanta Falcons,Falcons,TEAM,NFC,NCS,1900,Canton,Tom Benson Hall of Fame Stadium,OH,OUTDOOR
1,2019,PRE,1,2019080853,57838,08/08/2019,19:00:00,19:00:00,1565305200000,0610,...,Buffalo Bills,Bills,TEAM,AFC,ACE,3770,Orchard Park,New Era Field,NY,OUTDOOR
2,2019,PRE,1,2019080860,57846,08/08/2019,19:00:00,19:00:00,1565305200000,3410,...,New York Giants,Giants,TEAM,NFC,NCE,3757,East Rutherford,MetLife Stadium,NJ,OUTDOOR
3,2019,PRE,1,2019080852,57837,08/08/2019,19:30:00,19:30:00,1565307000000,0325,...,Baltimore Ravens,Ravens,TEAM,AFC,ACN,3695,Baltimore,M&T Bank Stadium,MD,OUTDOOR
4,2019,PRE,1,2019080856,57841,08/08/2019,19:30:00,19:30:00,1565307000000,1540,...,Detroit Lions,Lions,TEAM,NFC,NCN,2026,Detroit,Ford Field,MI,INDOOR


In [71]:
df.to_csv("../data/2019_schedule.csv", index=False)